In [1]:
from transformers import LineByLineTextDataset,DataCollatorForLanguageModeling, LlamaTokenizer,LlamaForCausalLM, LlamaTokenizer,TextDataset,DataCollatorWithPadding
from peft import get_peft_model, PeftModel, PeftConfig ,LoraConfig,TaskType
from transformers import TrainingArguments, Trainer
from datasets import Dataset , load_dataset
from torch.utils.data import Dataset, DataLoader
import torch


tokenizer = LlamaTokenizer.from_pretrained("/home/vicuna-weights-7B")
tokenizer.pad_token = tokenizer.eos_token
torch.set_default_dtype(torch.half)
model = LlamaForCausalLM.from_pretrained("/home/vicuna-weights-7B")

torch.set_default_dtype(torch.float)

/home/minigpt_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so
CUDA SETUP: CUDA runtime path found: /home/minigpt_env/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 113
CUDA SETUP: Loading binary /home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda113.so...


/home/minigpt_env/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/minigpt_env/lib/libcudart.so.11.0'), PosixPath('/home/minigpt_env/lib/libcudart.so')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.35s/it]


In [13]:
tokenizer = LlamaTokenizer.from_pretrained("/home/vicuna-weights-7B",add_eos_token=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id

In [2]:
from deepspeed.ops.adam import DeepSpeedCPUAdam
import deepspeed


In [3]:
#torch.cuda.empty_cache()


In [4]:
with open("medical_corpus.txt", "r", encoding="utf-8") as file:
    medical_text = file.read()

tokenized_text = tokenizer.encode(medical_text)
len(tokenized_text)

27716

In [5]:
block_size = 128
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/home/medical-llama/medical_corpus.txt",
    block_size=block_size,)

/home/minigpt_env/lib/python3.9/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
block_size = 128
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/home/medical-llama/medical_corpus.txt",
    block_size=block_size,)

In [6]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
) # handles next word prediction task

In [7]:
deepspeed_engine, optimizer, _, _ = deepspeed.initialize(model=model, config_params="/home/medical-llama/ds_config.json")

[2023-05-19 15:10:01,595] [INFO] [logging.py:96:log_dist] [Rank -1] DeepSpeed info: version=0.9.2, git-hash=unknown, git-branch=unknown
[2023-05-19 15:10:01,597] [INFO] [comm.py:606:init_distributed] Not using the DeepSpeed or dist launchers, attempting to detect MPI environment...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 deepspeed_engine, optimizer, _, _ = deepspeed.initialize(model=model, config_params="/ho     │
│   2                                                                                              │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/deepspeed/__init__.py:129 in initialize            │
│                                                                                                  │
│   126 │   global dist                                                                            │
│   127 │   from deepspeed import comm as dist                                                     │
│   128 │   dist_backend = get_accelerator().communication_backend_name()                          │
│ ❱ 129 │   dist.init_distributed(dist_backend=dist_backend, dist_init_required=dist_init_requir   │
│   130 │                                                                                          │
│   131 │   # Set config using config_params for backwards compat                                  │
│   132 │   if config is None and config_params is not None:                                       │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/deepspeed/comm/comm.py:612 in init_distributed     │
│                                                                                                  │
│   609 │   │   │   elif in_aws_sm():                                                              │
│   610 │   │   │   │   patch_aws_sm_env_for_torch_nccl_backend(verbose=verbose)                   │
│   611 │   │   │   else:                                                                          │
│ ❱ 612 │   │   │   │   mpi_discovery(distributed_port=distributed_port, verbose=verbose)          │
│   613 │   │                                                                                      │
│   614 │   │   if cdb is not None and cdb.is_initialized():                                       │
│   615 │   │   │   if int(os.getenv('RANK', '0')) == 0:                                           │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/deepspeed/comm/comm.py:631 in mpi_discovery        │
│                                                                                                  │
│   628 │   '''                                                                                    │
│   629 │   Discovery MPI environment via mpi4py and map to relevant dist state                    │
│   630 │   '''                                                                                    │
│ ❱ 631 │   from mpi4py import MPI                                                                 │
│   632 │   import subprocess                                                                      │
│   633 │   comm = MPI.COMM_WORLD                                                                  │
│   634 │   rank = comm.Get_rank()                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ModuleNotFoundError: No module named 'mpi4py'

In [8]:
training_args = TrainingArguments(
    
    output_dir="~/medical-llama/output-1",
    overwrite_output_dir=False,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    fp16=True,
    
)

In [9]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM,
 inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1,)
model = get_peft_model(model, peft_config).to('cuda:0')
model.print_trainable_parameters()

trainable params: 4194304 || all params: 6742618112 || trainable%: 0.06220586618327525


In [ ]:
tokenizer.pad_token_id = 0

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
     
)

In [11]:
trainer.train()

/home/minigpt_env/lib/python3.9/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: deelipvenkat. Use `wandb login --relogin` to force relogin


Step,Training Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.train()                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/transformers/trainer.py:1664 in train              │
│                                                                                                  │
│   1661 │   │   inner_training_loop = find_executable_batch_size(                                 │
│   1662 │   │   │   self._inner_training_loop, self._train_batch_size, args.auto_find_batch_size  │
│   1663 │   │   )                                                                                 │
│ ❱ 1664 │   │   return inner_training_loop(                                                       │
│   1665 │   │   │   args=args,                                                                    │
│   1666 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1667 │   │   │   trial=trial,                                                                  │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/transformers/trainer.py:1932 in                    │
│ _inner_training_loop                                                                             │
│                                                                                                  │
│   1929 │   │   │   │   │   with model.no_sync():                                                 │
│   1930 │   │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                  │
│   1931 │   │   │   │   else:                                                                     │
│ ❱ 1932 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1933 │   │   │   │                                                                             │
│   1934 │   │   │   │   if (                                                                      │
│   1935 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/transformers/trainer.py:2718 in training_step      │
│                                                                                                  │
│   2715 │   │   │   loss = loss / self.args.gradient_accumulation_steps                           │
│   2716 │   │                                                                                     │
│   2717 │   │   if self.do_grad_scaling:                                                          │
│ ❱ 2718 │   │   │   self.scaler.scale(loss).backward()                                            │
│   2719 │   │   elif self.use_apex:                                                               │
│   2720 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2721 │   │   │   │   scaled_loss.backward()                                                    │
│                                                                                                  │
│ /home/minigpt_env/lib/python3.9/site-packages/torch/_tensor.py:487 in backward                   │
│                                                                                                  │
│    484 │   │   │   │   create_graph=create_graph,          

In [ ]:
# saving the fine tuned model
# model.save_pretrained("~/medical-llama/saved_models")

In [ ]:
# pushing the peft weights to the hub
#notebook_login()
#model.push_to_hub("my_awesome_peft_model")